In [1]:
initialize = True

In [2]:
import ipyfan
import numpy as np
import ipywidgets as widgets

In [3]:
import iio
import glob
import rasterio

In [4]:
import os
from natsort import natsorted
import scipy
import scipy.ndimage

In [5]:
# Dictionary containing to paths to the different data useful for annotation
main_paths = '/home/thibaud/Documents/PhD/dataset/kayrros/v6/*/B11B12_change/*.tif'
im_options = {'methane': ('B11B12_change', 'B11B12_change'),
           'nomethane': ('B11B12_change', 'B11B08_change'),
             'rgb': ('B11B12_change', 'rgb')}
ref_options = {'kayrros': ('B11B12_change', 'labels')}
out = ('B11B12_change', 'labels_v2')

# FIXME Utiliser un fichier de config pour gérer tout ça?

In [6]:
list_paths = natsorted(glob.glob(main_paths))

In [7]:
list_paths

['/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2019-05-10_08:46:01.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2020-05-04_08:46:01.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2021-03-10_08:48:01.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2021-03-15_08:47:19.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2021-04-24_08:45:59.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2021-05-14_08:45:59.tif',
 '/home/thibaud/Documents/PhD/dataset/kayrros/v6/0bajpjoye1p85kd3mw3jccy6pxz6p1gy_Obaiyed_Gas_Plant_002/B11B12_change/2021-05-24_08:45:5

In [8]:
len(list_paths)

1092

In [9]:
# This function transform the generic input image into a uint8 image for visualization onto the canvas
def visu(im):
    p=0.001
    mi, ma = np.quantile(im, [p, 1-p], axis=(0,1), keepdims=True)
    im = im.clip(mi, ma)
    im = (im-mi) / (ma-mi)
    return im*255.

def scale(im, shape):
    if im.shape[:2] == shape[:2]:
        return im
    zoom = [s2/s1 for s1, s2 in zip(im.shape[:2], shape[:2])]
    if len(im.shape) == 3:
        zoom += [1]
    return scipy.ndimage.zoom(im, zoom=zoom)
    

def read_imgs(path, options):
    #ims = [iio.read(path.replace(opt[0], opt[1])) for key, opt in options.items()]
    ims = [rasterio.open(path.replace(opt[0], opt[1])).read().transpose(1,2,0) for key, opt in options.items()]
    ims = [im[:,:,0] if im.shape[2] < 3 else im[:,:,:3] if im.shape[2] > 3 else im for im in ims]
    # Scale all images so they have the same shape, that way we can switch between them on the canvas
    ims = [scale(im, ims[0].shape) for im in ims]
    return ims

def create_ref(path, options):
    # TODO allow for multiple options
    opt = next(iter(options.values()))
    path = path.replace(opt[0], opt[1])
    if os.path.isfile(path):
        im = iio.read(path)
        ref = (im[:,:,0] > 0.)*1.
        ref = np.stack((255*np.ones(ref.shape), 100*np.ones(ref.shape), 0*np.ones(ref.shape), 255*np.ones(ref.shape)), axis=2) * ref[:,:,None]
        return [ref], im
    else:
        return [None], None


In [10]:
# Only run this cell once. This prevent reseting the state of the annotation tool by mistake when modifying something else
if initialize:
    
    show_mask = True
    
    validate_button = widgets.Button(
        description='Validate',
        disabled=False
    )
    reset_button = widgets.Button(
        description='Reset',
        disabled=False)
    switchMask_button = widgets.Button(
        description='Hide/show ref mask',
        disabled=False)
    
    selectMask_button = widgets.Button(
        description='Select ref mask',
        disabled=False)
    
    previous_button = widgets.Button(
        description='Previous',
        disabled=False)

    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(list_paths)-1,
        description='Progress:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': 'black'},
        orientation='horizontal'
    )

    def validate(b):
        global w, show_mask
        out_path = list_paths[progress.value].replace(out[0], out[1])
        # Check that the folder exists
        if not os.path.isdir(os.path.dirname(out_path)):
            os.makedirs(os.path.dirname(out_path))
        if np.sum(w.data[:,:,1]) > 0:
            iio.write(out_path, w.data[:,:,1])
        else:
            # Annotation is empty, delete the corresponding file if it exists
            if os.path.exists(out_path):
                os.remove(out_path)

        progress.value += 1

        global curr_ims, curr_ref, ref
        curr_ims = read_imgs(list_paths[progress.value], im_options)
        curr_ref, ref = create_ref(list_paths[progress.value], ref_options)
        w.reset(visu(curr_ims[0]), curr_ref[0] if show_mask else None)
        
        print(progress.value)
    
    def previous(b):
        global w, show_mask
        progress.value -= 1

        global curr_ims, curr_ref, ref
        curr_ims = read_imgs(list_paths[progress.value], im_options)
        curr_ref, ref = create_ref(list_paths[progress.value], ref_options)
        w.reset(visu(curr_ims[0]), curr_ref[0] if show_mask else None)
        
    def select_mask(b):
        global w, show_mask, ref
        out_path = list_paths[progress.value].replace(out[0], out[1])
        # Check that the folder exists
        if not os.path.isdir(os.path.dirname(out_path)):
            os.makedirs(os.path.dirname(out_path))
        if np.sum(ref) > 0:
            iio.write(out_path, ref)
        else:
            # Annotation is empty, delete the corresponding file if it exists
            if os.path.exists(out_path):
                os.remove(out_path)

        progress.value += 1

        global curr_ims, curr_ref
        curr_ims = read_imgs(list_paths[progress.value], im_options)
        curr_ref, ref = create_ref(list_paths[progress.value], ref_options)
        w.reset(visu(curr_ims[0]), curr_ref[0] if show_mask else None)

    def reset(b):
        global w, show_mask
        w.reset(visu(curr_ims[0]), curr_ref[0] if show_mask else None)
        
    def switch_mask(b):
        global show_mask
        show_mask = not show_mask

    validate_button.on_click(validate)
    reset_button.on_click(reset)
    switchMask_button.on_click(switch_mask)
    selectMask_button.on_click(select_mask)
    previous_button.on_click(previous)
    imChoiceButtons = [widgets.Button(description=option, disabled=False) for option in im_options.keys()]
    [button.on_click(lambda b, i=i: w.set_image(visu(curr_ims[i]), curr_ref[0] if show_mask else None)) for i, button in enumerate(imChoiceButtons)]
    initialize = False

In [11]:
curr_ims = read_imgs(list_paths[progress.value], im_options)
curr_ref, ref = create_ref(list_paths[progress.value], ref_options)

w = ipyfan.segmenter()
w.layout.width = '700px'
w.layout.height = '700px'
w.set_image(visu(curr_ims[0]), curr_ref[0])


widgets.VBox([widgets.HBox([w, widgets.VBox(imChoiceButtons + [switchMask_button])]), widgets.HBox([validate_button, reset_button, selectMask_button, previous_button]), progress])